## Демидова Екатерина

# Задание 1 (5 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

In [1]:
import re

In [2]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
from sklearn.metrics.pairwise import cosine_distances
from sentence_transformers import SentenceTransformer

In [8]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')[:100] # сокращаю для ускорения работы
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

In [6]:
def jaccard(definitions, context):
    context_words = set(context.split())
    best_score = 0
    best_definition = None

    for definition in definitions:
        definition_words = set(definition.split())
        intersection = definition_words & context_words
        union = definition_words | context_words
        score = len(intersection) / len(union) if len(union) > 0 else 0

        if score > best_score:
            best_score = score
            best_definition = definition

    return best_definition

In [7]:
definition_embeddings_cache = {}

def cosine_distance(definitions, context):
    context_emb = model.encode(context)
    best_score = 1
    best_definition = None

    for definition in definitions:
        if definition not in definition_embeddings_cache:
            definition_emb = model.encode(definition)
            definition_embeddings_cache[definition] = definition_emb
        else:
            definition_emb = definition_embeddings_cache[definition]

        score = cosine_distances(context_emb.reshape(1, -1), definition_emb.reshape(1, -1))[0][0]

        if score < best_score:
            best_score = score
            best_definition = definition

    return best_definition

In [8]:
definitions_cache = {}

def find_definitions(lemma):
    if lemma in definitions_cache:
        return definitions_cache[lemma]

    definitions = []
    for synset in wn.synsets(lemma):
        definitions.append(synset.definition())
    definitions_cache[lemma] = definitions
    return definitions

In [10]:
all_answers = 0
right_jaccard = 0
right_cosine = 0

for sent in corpus_wsd:
    full_sent = ' '.join([x[2] for x in sent])
    clean_sent = re.sub(r'[^\w\s]', '', full_sent.lower()).strip()

    for word_info in sent:
        tag = word_info[0]
        lemma = word_info[1]

        if tag != '':
            actual_meaning = wn.lemma_from_key(tag).synset().definition()
            all_meanings = find_definitions(lemma)

            all_answers += 1

            prediction_jaccard = jaccard(all_meanings, clean_sent)
            if actual_meaning == prediction_jaccard:
                right_jaccard += 1

            prediction_cosine = cosine_distance(all_meanings, clean_sent)
            if actual_meaning == prediction_cosine:
                right_cosine += 1

accuracy_jaccard = right_jaccard / all_answers
accuracy_cosine = right_cosine / all_answers

print(f'Accuracy for Jaccard score: {accuracy_jaccard:.4f}')
print(f'Accuracy for Cosine distance: {accuracy_cosine:.4f}')

Accuracy for Jaccard score: 0.2453
Accuracy for Cosine distance: 0.4428


# Задание 2 (5 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты.

In [2]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv', sep='\t')

grouped_df = df.groupby('word')[['word', 'context', 'gold_sense_id']]

In [4]:
from sklearn.cluster import KMeans, DBSCAN, AffinityPropagation, AgglomerativeClustering
import numpy as np
from sklearn.metrics import adjusted_rand_score

from IPython.display import Image
from IPython.core.display import HTML

**DBSCAN**

In [13]:
ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].values
    X = np.zeros((len(texts), 768))
    for i, text in enumerate(texts):
        X[i] = model.encode(text)

    cluster = DBSCAN(min_samples=10, eps=0.9)

    cluster.fit(X)
    labels = np.array(cluster.labels_)+1

    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))

print(np.mean(ARI))

-0.0013973419450999875


**Эксперимент №1**

*   DBSCAN(min_samples=1, eps=0.1)
*   Усредненный ARI = 0.001053019960000099
*   Время выполнения -- 10 минут

---

**Эксперимент №2**

*   DBSCAN(min_samples=5, eps=0.5)
*   Усредненный ARI = -0.011271692824715207
*   Время выполнения -- 11 минут

---

**Эксперимент №3**

*   DBSCAN(min_samples=8, eps=0.3)
*   Усредненный ARI = 0.0
*   Время выполнения -- 9 минут

---

**Эксперимент №4**

*   DBSCAN(min_samples=4, eps=0.8)
*   Усредненный ARI = -0.001784882962278153
*   Время выполнения -- 10 минут

---

**Эксперимент №5**

*   DBSCAN(min_samples=10, eps=0.9)
*   Усредненный ARI = -0.0013973419450999875
*   Время выполнения -- 9 минут

**Affinity Propagation**

In [21]:
ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].values
    X = np.zeros((len(texts), 768))
    for i, text in enumerate(texts):
        X[i] = model.encode(text)

    cluster = AffinityPropagation(damping=0.8, preference=-100)

    cluster.fit(X)
    labels = np.array(cluster.labels_)+1

    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))

print(np.mean(ARI))

0.0


**Эксперимент №1**

*   AffinityPropagation(damping=0.9)
*   Усредненный ARI = 0.05297560306165972
*   Время выполнения -- 10 минут

---

**Эксперимент №2**

*   AffinityPropagation(damping=0.9, max_iter=100)
*   Усредненный ARI = 0.05297560306165972
*   Время выполнения -- 10 минут

---

**Эксперимент №3**

*   AffinityPropagation(damping=0.5)
*   Усредненный ARI = 0.042740969848549505
*   Время выполнения -- 10 минут

---

**Эксперимент №4**

*   AffinityPropagation(damping=0.8, preference=-10)
*   Усредненный ARI = 0.0
*   Время выполнения -- 10 минут

---

**Эксперимент №5**

*   AffinityPropagation(damping=0.8, preference=-100)
*   Усредненный ARI = 0.0
*   Время выполнения -- 10 минут

**Agglomerative clustering**

In [27]:
ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].values
    X = np.zeros((len(texts), 768))
    for i, text in enumerate(texts):
        X[i] = model.encode(text)

    cluster = AgglomerativeClustering(n_clusters=20)

    cluster.fit(X)
    labels = np.array(cluster.labels_)+1

    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))

print(np.mean(ARI))

0.04406294885576405


**Эксперимент №1**

*   AgglomerativeClustering(n_clusters=2)
*   Усредненный ARI = -0.011976265536517934
*   Время выполнения -- 10 минут

---

**Эксперимент №2**

*   AgglomerativeClustering(n_clusters=5)
*   Усредненный ARI = 0.03379296274962468
*   Время выполнения -- 9 минут

---

**Эксперимент №3**

*   AgglomerativeClustering(n_clusters=8)
*   Усредненный ARI = 0.057641599566266404
*   Время выполнения -- 9 минут

---

**Эксперимент №4**

*   AgglomerativeClustering(n_clusters=10)
*   Усредненный ARI = 0.05972280261375722
*   Время выполнения -- 9 минут

---

**Эксперимент №5**

*   AgglomerativeClustering(n_clusters=20)
*   Усредненный ARI = 0.04406294885576405
*   Время выполнения -- 10 минут

**K-Means**

In [32]:
ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].values
    X = np.zeros((len(texts), 768))
    for i, text in enumerate(texts):
        X[i] = model.encode(text)

    cluster = KMeans(3, max_iter=150)

    cluster.fit(X)
    labels = np.array(cluster.labels_)+1

    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))

print(np.mean(ARI))

0.10958832689053119


**Эксперимент №1**

*   KMeans(2)
*   Усредненный ARI = 0.08833864849790815
*   Время выполнения -- 10 минут

---

**Эксперимент №2**

*   KMeans(5)
*   Усредненный ARI = 0.05055906241628728
*   Время выполнения -- 10 минут

---

**Эксперимент №3**

*   KMeans(2, max_iter=600)
*   Усредненный ARI = 0.032885409100014154
*   Время выполнения -- 10 минут

---

**Эксперимент №4**

*   KMeans(4, max_iter=200)
*   Усредненный ARI = 0.0918641433254593
*   Время выполнения -- 10 минут

---

**Эксперимент №5**

*   KMeans(3, max_iter=150)
*   Усредненный ARI = 0.10958832689053119
*   Время выполнения -- 10 минут


Последний результат самый удачный за все попытки. Время выполнения везде примерно одинаковое.

**Bisecting K-Means**

Сначала все данные воспринимаются как один большой кластер, затем он делится на 2 кластера, и так далее, пока не получится нужное число кластеров. Более последовательный подход к кластеризации, чем простой K-Means, подходит для иерархических данных.

In [33]:
from sklearn.cluster import BisectingKMeans

In [41]:
ARI = []

for key, _ in grouped_df:
    texts = grouped_df.get_group(key)['context'].values
    X = np.zeros((len(texts), 768))
    for i, text in enumerate(texts):
        X[i] = model.encode(text)

    cluster = BisectingKMeans(n_clusters=10, max_iter=100)

    cluster.fit(X)
    labels = np.array(cluster.labels_)+1

    ARI.append(adjusted_rand_score(grouped_df.get_group(key)['gold_sense_id'], labels))

print(np.mean(ARI))

0.05207745839226895


**Эксперимент №1**

*   BisectingKMeans(n_clusters=6, max_iter=200)
*   Усредненный ARI = 0.035013982964821305
*   Время выполнения -- 10 минут

---

**Эксперимент №2**

*   BisectingKMeans(n_clusters=4, max_iter=100)
*   Усредненный ARI = 0.06996833639802548
*   Время выполнения -- 10 минут

---

**Эксперимент №3**

*   BisectingKMeans(n_clusters=3, max_iter=100)
*   Усредненный ARI = 0.035782611446943856
*   Время выполнения -- 10 минут

---

**Эксперимент №4**

*   BisectingKMeans(n_clusters=5, max_iter=150)
*   Усредненный ARI = 0.07665231909144522
*   Время выполнения -- 10 минут

---

**Эксперимент №5**

*   BisectingKMeans(n_clusters=10, max_iter=100)
*   Усредненный ARI = 0.05207745839226895
*   Время выполнения -- 10 минут